In [3]:
from twitter import Twitter, OAuth, TwitterHTTPError
import tweepy
import time
import random
import os


# put your tokens, keys, secrets, and Twitter handle in the following variables

CONSUMER_KEY = 'zl2ge4yWyhbWk8ut5DEbBUpEP'
CONSUMER_SECRET = 'Rt4WghVKi93cY54YdnzQh9tHJegylFujIajkZfcWzRuGKpk0aF'
OAUTH_TOKEN = '345639711-yxVH5s2BO3qunZ1m6MZpC0IvC68Rd3OsywkOoSgH'
OAUTH_SECRET = '3aQqV2ZrrTQ8dJqQid9JMgLjgn0mjQKzIemmFn4oFgqa8'

# put your handle here
TWITTER_HANDLE = "senorsnipes"


# put the full path and file name of the file you want to store your "already followed" list in
#ALREADY_FOLLOWED_FILE = "Users/jackdwyer/Desktop/projects/ALREADY_FOLLOWED_FILE.txt"


#For twitter
t = Twitter(auth=OAuth(OAUTH_TOKEN, OAUTH_SECRET,
            CONSUMER_KEY, CONSUMER_SECRET))


#For tweepy
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_SECRET)
# Creation of the actual interface, using authentication
api = tweepy.API(auth)





def search_tweets(q, count=100, result_type="recent"):
    """
        Returns a list of tweets matching a certain phrase (hashtag, word, etc.)
    """

    return t.search.tweets(q=q, result_type=result_type, count=count)


def auto_fav(q, count=100, result_type="recent"):
    """
        Favorites tweets that match a certain phrase (hashtag, word, etc.)
    """

    result = search_tweets(q, count, result_type)

    for tweet in result["statuses"]:
        try:
            # don't favorite your own tweets
            if tweet["user"]["screen_name"] == TWITTER_HANDLE:
                continue

            result = t.favorites.create(_id=tweet["id"])
            print("favorited: %s" % (result["text"].encode("utf-8")))

        # when you have already favorited a tweet, this error is thrown
        except TwitterHTTPError as e:
            print("error: %s" % (str(e)))


def auto_rt(q, count=0, result_type="recent"):
    """
        Retweets tweets that match a certain phrase (hashtag, word, etc.)
    """

    result = search_tweets(q, count, result_type)

    for tweet in result["statuses"]:
        try:
            # don't retweet your own tweets
            if tweet["user"]["screen_name"] == TWITTER_HANDLE:
                continue

            result = t.statuses.retweet(id=tweet["id"])
            print("retweeted: %s" % (result["text"].encode("utf-8")))

        # when you have already retweeted a tweet, this error is thrown
        except TwitterHTTPError as e:
            print("error: %s" % (str(e)))


def auto_follow(q, count=100, result_type="recent"):
    """
        Follows anyone who tweets about a specific phrase (hashtag, word, etc.)
    """

    result = search_tweets(q, count, result_type)
    following = set(t.friends.ids(screen_name=TWITTER_HANDLE)["ids"])

    # make sure the "already followed" file exists
    #if not os.path.isfile(ALREADY_FOLLOWED_FILE):
    #    with open(ALREADY_FOLLOWED_FILE, "w") as out_file:
    #        out_file.write("")

        # read in the list of user IDs that the bot has already followed in the
        # past
    #do_not_follow = set()
    #dnf_list = []
    #with open(ALREADY_FOLLOWED_FILE) as in_file:
    #    for line in in_file:
    #        dnf_list.append(int(line))

    #do_not_follow.update(set(dnf_list))
    #del dnf_list

    for tweet in result["statuses"]:
        try:
            if (tweet["user"]["screen_name"] != TWITTER_HANDLE and
                    tweet["user"]["id"] not in following):

                t.friendships.create(user_id=tweet["user"]["id"], follow=True)
                t.mutes.users.create(user_id=tweet["user"]["id"])
                following.update(set([tweet["user"]["id"]]))

                print("followed %s" % (tweet["user"]["screen_name"]))

        except TwitterHTTPError as e:
            print("error: %s" % (str(e)))

            # quit on error unless it's because someone blocked me
            if "blocked" not in str(e).lower():
                quit()
    '''following = set(t.friends.ids(screen_name=TWITTER_HANDLE)["ids"])
    muted = set(t.mutes.users.ids(screen_name=TWITTER_HANDLE)["ids"])

    not_muted = following - muted

                # put user IDs of people you do not want to mute here
    users_keep_unmuted = set([])

    # mute all
    for user_id in not_muted:
        if user_id not in users_keep_unmuted:
            t.mutes.users.create(user_id=user_id)
            print("muted %d" % (user_id))
'''
def auto_follow_followers():
    """
        Follows back everyone who's followed you
    """

    following = set(t.friends.ids(screen_name=TWITTER_HANDLE)["ids"])
    followers = set(t.followers.ids(screen_name=TWITTER_HANDLE)["ids"])

    not_following_back = followers - following

    for user_id in not_following_back:
        try:
            t.friendships.create(user_id=user_id, follow=True)
            t.mutes.users.create(user_id=user_id)

        except Exception as e:
            print("error: %s" % (str(e)))
"""
def auto_unfollow_nonfollowers(unfollow_number):
    

    following = set(t.friends.ids(screen_name=TWITTER_HANDLE)["ids"])
    followers = set(t.followers.ids(screen_name=TWITTER_HANDLE)["ids"])

    # put user IDs here that you want to keep following even if they don't
    # follow you back
    users_keep_following = set([])

    not_following_back = following - followers

    choose_unfollow = random.sample( not_following_back, unfollow_number )

    # make sure the "already followed" file exists
    if not os.path.isfile(ALREADY_FOLLOWED_FILE):
        with open(ALREADY_FOLLOWED_FILE, "w") as out_file:
            out_file.write("")

    # update the "already followed" file with users who didn't follow back
    already_followed = set(choose_unfollow)
    af_list = []
    with open(ALREADY_FOLLOWED_FILE) as in_file:
        for line in in_file:
            af_list.append(int(line))

    already_followed.update(set(af_list))
    del af_list

    with open(ALREADY_FOLLOWED_FILE, "w") as out_file:
        for val in already_followed:
            out_file.write(str(val) + "\n")

    for user_id in choose_unfollow:
        if user_id not in users_keep_following:
            t.friendships.destroy(user_id=user_id)
            print("unfollowed %d" % (user_id))
"""

def auto_mute_following():
    """
        Mutes everyone that you are following
    """
    following = set(t.friends.ids(screen_name=TWITTER_HANDLE)["ids"])
    muted = set(t.mutes.users.ids(screen_name=TWITTER_HANDLE)["ids"])

    not_muted = following - muted

    # put user IDs of people you do not want to mute here
    users_keep_unmuted = set([])

    # mute all
    for user_id in not_muted:
        if user_id not in users_keep_unmuted:
            t.mutes.users.create(user_id=user_id)
            print("muted %d" % (user_id))


def auto_unmute():
    """
        Unmutes everyone that you have muted
    """
    muted = set(t.mutes.users.ids(screen_name=TWITTER_HANDLE)["ids"])

    # put user IDs of people you want to remain muted here
    users_keep_muted = set([])

    # mute all
    for user_id in muted:
        if user_id not in users_keep_muted:
            t.mutes.users.destroy(user_id=user_id)
            print("unmuted %d" % (user_id))


def show_performance():
    user = api.get_user("@Data88Geek")
    print( "User Name: %s" %user.screen_name )
    print( "Follow You: %s" %user.followers_count )
    print( "You Follow: %s" %user.friends_count )
    print( "Ratio: %s" %float(user.followers_count * 100 /user.friends_count ) )
    print( "Tweets: %s" %user.statuses_count )
    print( "Favorites: %s" %user.favourites_count )
# performance matrix includes name, number of followers, number of users you followed, ratio, number of tweets, number of favorites

"""
def control_ratio(ratio):
    user = api.get_user("@Data88Geek")
    if( float(user.followers_count * 100 / user.friends_count ) < ratio ):
        auto_unfollow_nonfollowers(unfollow_number=5)
# you cna set "follower to followed" ratio here


try:
    while True:
        print "Start : %s" % time.ctime() # -> starting time
        
        print("Here we go!")

        # target list( ex: I am interested in data and fintech related topic )
        target = [ "#DataScience", "#DataAnalysis", "#MachineLearning", "#DataMining", "#NLP",
                   "#Python", "#BigData", "#Hadoop", "#MapReduce", "#PredictiveModeling",
                   "#Google", "#FaceBook", "#Twitter", "#Palantir", "#BitCoin" ]
                   
        run_target = random.sample( target, 3 ) # -> randomly get 3 targets from the target list
        
        # for the chosen targets, we do the following actions
        for i in run_target:
            auto_follow(i, count=3)
            auto_fav(i, count=5)
            auto_rt(i, count=5)

        control_ratio(ratio=200) # -> default ratio is 200, aka number followers is 2 times of number of users you followed
        
        # the following three function are not using in defult
        #auto_follow_followers()
        #auto_mute_following()
        #auto_unmute()

        print( "Cool man! You finish a twitter hack!" )
        print("")
        
        #You can see your updated performance matrix here
        print( "Your performance:" )
        show_performance()
        print("")
        print( "Let's take a break" )
        
        print "End : %s" % time.ctime() # -> ending time
        print("")
        print("-------------------------------------------------------------------------------------------------------")
        print("")
        
        time.sleep(300) # -> default cycle is 5 mins( 300 seconds )

# if anything go wrong
except KeyboardInterrupt:
    print("OOPS! Something wrong!")
"""    




def unfollow():
    followers = api.followers_ids('senorsnipes')
    friends = api.friends_ids('senorsnipes')
 
    for f in friends:
        if f not in followers:
            api.destroy_friendship(f)
#unfollow()
auto_follow('streamer')



followed LifegivaRadio
followed Gabrielkuz1
followed EPEGames
followed Que___B
followed aini_semok
followed 5don_KUN
followed CatsWhiskersTV
followed mytruthaboutAA
followed kayPOW
followed wilpat34
followed Spacemouse65
followed Armydude206
followed familylifenypa
followed drtalukdard
followed mchavez6789
followed Flying_Baepsae
followed FromThe70sguy
followed fresh_005
followed ccabexllo
followed Pindalicious
followed northernangel91
followed Matandgo
followed themegakersh
followed Adrian_Duncan90
followed Selmilipa
followed streamsupport5
followed progorexne1977
followed usmc40471
followed exoo_loverss
followed NBCSWizards
followed fifthchele
followed RAIRADIOLIVE
followed TwitchHispano
followed OfficialCyastic
followed RADIOPUSHERSTV
followed 995SOUTHBEACH
followed 1061SANDIEGO
followed HBAWhiteyWhite
followed OnlinethreeTV2
followed TerpsWatch
followed Ibtihal40365423
followed FingerLakes1
followed Spidrh_
followed cwmilaroses
followed bblingspot
followed mzmz_moatz
followed Jimel

In [ ]:
unfollow()